In [1]:
!pip install pyspark==3.3.4
!pip install PyDeequ==1.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 1.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.4-py2.py3-none-any.whl size=281945735 sha256=0668c052d96c506c1903dcd2e11c4d77119fc192001eb12f11e1109649565313
  Stored in directory: /root/.cache/pip/wheels/33/e6/00/f21ce51bdac1b8689263d156ffd3d300558ec0b452945f7ee4
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [38]:
import os

import pydeequ
from pydeequ.analyzers import *
from pydeequ.verification import *
from pydeequ.profiles import *
from pydeequ.checks import *
from pydeequ.verification import *

from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import regexp_replace, expr

In [9]:
os.environ['SPARK_VERSION'] = '3.3'

In [10]:
spark = (
    SparkSession.builder \
    .config('spark.jars.packages', pydeequ.deequ_maven_coord) \
    .config('spark.jars.excludes', pydeequ.f2j_maven_coord) \
    .getOrCreate()
)

In [11]:
spark

In [12]:
df = spark.read \
  .option('header', 'true') \
  .option('delimiter', ';') \
  .option('encoding', 'UTF16') \
  .csv('/content/bilheteria-diaria-obras-por-distribuidoras-2023-12.csv')

root
 |-- DATA_EXIBICAO: string (nullable = true)
 |-- TITULO_ORIGINAL: string (nullable = true)
 |-- TITULO_BRASIL: string (nullable = true)
 |-- CPB_ROE: string (nullable = true)
 |-- PAIS_OBRA: string (nullable = true)
 |-- REGISTRO_SALA: string (nullable = true)
 |-- NOME_SALA: string (nullable = true)
 |-- PUBLICO: string (nullable = true)
 |-- REGISTRO_GRUPO_EXIBIDOR: string (nullable = true)
 |-- REGISTRO_EXIBIDOR: string (nullable = true)
 |-- REGISTRO_COMPLEXO: string (nullable = true)
 |-- MUNICIPIO_SALA_COMPLEXO: string (nullable = true)
 |-- UF_SALA_COMPLEXO: string (nullable = true)
 |-- RAZAO_SOCIAL_DISTRIBUIDORA: string (nullable = true)
 |-- CNPJ_DISTRIBUIDORA�: string (nullable = true)



In [13]:
df.show()

+-------------+--------------------+-------------------+--------------+--------------+-------------+--------------------+-------+-----------------------+-----------------+-----------------+-----------------------+----------------+--------------------------+-------------------+
|DATA_EXIBICAO|     TITULO_ORIGINAL|      TITULO_BRASIL|       CPB_ROE|     PAIS_OBRA|REGISTRO_SALA|           NOME_SALA|PUBLICO|REGISTRO_GRUPO_EXIBIDOR|REGISTRO_EXIBIDOR|REGISTRO_COMPLEXO|MUNICIPIO_SALA_COMPLEXO|UF_SALA_COMPLEXO|RAZAO_SOCIAL_DISTRIBUIDORA|CNPJ_DISTRIBUIDORA�|
+-------------+--------------------+-------------------+--------------+--------------+-------------+--------------------+-------+-----------------------+-----------------+-----------------+-----------------------+----------------+--------------------------+-------------------+
|   01/12/2023|ABELHA GIRL A SUP...|               null|B2300132300000|        BRASIL|         null|luciano da luz mo...|    350|                   null|             

In [14]:
df = df.withColumnRenamed(df.columns[-1], df.columns[-1][:-1])

In [15]:
df.show()

+-------------+--------------------+-------------------+--------------+--------------+-------------+--------------------+-------+-----------------------+-----------------+-----------------+-----------------------+----------------+--------------------------+-------------------+
|DATA_EXIBICAO|     TITULO_ORIGINAL|      TITULO_BRASIL|       CPB_ROE|     PAIS_OBRA|REGISTRO_SALA|           NOME_SALA|PUBLICO|REGISTRO_GRUPO_EXIBIDOR|REGISTRO_EXIBIDOR|REGISTRO_COMPLEXO|MUNICIPIO_SALA_COMPLEXO|UF_SALA_COMPLEXO|RAZAO_SOCIAL_DISTRIBUIDORA| CNPJ_DISTRIBUIDORA|
+-------------+--------------------+-------------------+--------------+--------------+-------------+--------------------+-------+-----------------------+-----------------+-----------------+-----------------------+----------------+--------------------------+-------------------+
|   01/12/2023|ABELHA GIRL A SUP...|               null|B2300132300000|        BRASIL|         null|luciano da luz mo...|    350|                   null|             

In [16]:
df = df.withColumn('CNPJ_DISTRIBUIDORA', expr('substring(CNPJ_DISTRIBUIDORA, 1, length(CNPJ_DISTRIBUIDORA)-1)'))

In [17]:
df.show(2)

+-------------+--------------------+-------------------+--------------+--------------+-------------+--------------------+-------+-----------------------+-----------------+-----------------+-----------------------+----------------+--------------------------+------------------+
|DATA_EXIBICAO|     TITULO_ORIGINAL|      TITULO_BRASIL|       CPB_ROE|     PAIS_OBRA|REGISTRO_SALA|           NOME_SALA|PUBLICO|REGISTRO_GRUPO_EXIBIDOR|REGISTRO_EXIBIDOR|REGISTRO_COMPLEXO|MUNICIPIO_SALA_COMPLEXO|UF_SALA_COMPLEXO|RAZAO_SOCIAL_DISTRIBUIDORA|CNPJ_DISTRIBUIDORA|
+-------------+--------------------+-------------------+--------------+--------------+-------------+--------------------+-------+-----------------------+-----------------+-----------------+-----------------------+----------------+--------------------------+------------------+
|   01/12/2023|ABELHA GIRL A SUP...|               null|B2300132300000|        BRASIL|         null|luciano da luz mo...|    350|                   null|             nul

In [18]:
df.printSchema()

root
 |-- DATA_EXIBICAO: string (nullable = true)
 |-- TITULO_ORIGINAL: string (nullable = true)
 |-- TITULO_BRASIL: string (nullable = true)
 |-- CPB_ROE: string (nullable = true)
 |-- PAIS_OBRA: string (nullable = true)
 |-- REGISTRO_SALA: string (nullable = true)
 |-- NOME_SALA: string (nullable = true)
 |-- PUBLICO: string (nullable = true)
 |-- REGISTRO_GRUPO_EXIBIDOR: string (nullable = true)
 |-- REGISTRO_EXIBIDOR: string (nullable = true)
 |-- REGISTRO_COMPLEXO: string (nullable = true)
 |-- MUNICIPIO_SALA_COMPLEXO: string (nullable = true)
 |-- UF_SALA_COMPLEXO: string (nullable = true)
 |-- RAZAO_SOCIAL_DISTRIBUIDORA: string (nullable = true)
 |-- CNPJ_DISTRIBUIDORA: string (nullable = true)



In [19]:
df = df.withColumn('PUBLICO', df['PUBLICO'].cast('integer'))

In [20]:
df.printSchema()

root
 |-- DATA_EXIBICAO: string (nullable = true)
 |-- TITULO_ORIGINAL: string (nullable = true)
 |-- TITULO_BRASIL: string (nullable = true)
 |-- CPB_ROE: string (nullable = true)
 |-- PAIS_OBRA: string (nullable = true)
 |-- REGISTRO_SALA: string (nullable = true)
 |-- NOME_SALA: string (nullable = true)
 |-- PUBLICO: integer (nullable = true)
 |-- REGISTRO_GRUPO_EXIBIDOR: string (nullable = true)
 |-- REGISTRO_EXIBIDOR: string (nullable = true)
 |-- REGISTRO_COMPLEXO: string (nullable = true)
 |-- MUNICIPIO_SALA_COMPLEXO: string (nullable = true)
 |-- UF_SALA_COMPLEXO: string (nullable = true)
 |-- RAZAO_SOCIAL_DISTRIBUIDORA: string (nullable = true)
 |-- CNPJ_DISTRIBUIDORA: string (nullable = true)



## Analysers

In [26]:
analyse = (
    AnalysisRunner(spark)
    .onData(df)
    .addAnalyzer(Size())
    .addAnalyzer(Completeness('NOME_SALA'))
    .addAnalyzer(Completeness('PUBLICO'))
    .addAnalyzer(Mean('PUBLICO'))
    .addAnalyzer(CountDistinct('MUNICIPIO_SALA_COMPLEXO'))
    .addAnalyzer(Compliance('PUBLICO', 'PUBLICO > 10'))
    .run()
)

In [28]:
result_df = AnalyzerContext.successMetricsAsDataFrame(spark, analyse)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


DataFrame[entity: string, instance: string, name: string, value: double]

In [29]:
result_df.show()

+-------+--------------------+-------------+------------------+
| entity|            instance|         name|             value|
+-------+--------------------+-------------+------------------+
| Column|             PUBLICO| Completeness|               1.0|
| Column|             PUBLICO|         Mean|44.088970640254686|
|Dataset|                   *|         Size|          141350.0|
| Column|             PUBLICO|   Compliance|0.6735267067562788|
| Column|MUNICIPIO_SALA_CO...|CountDistinct|             443.0|
| Column|           NOME_SALA| Completeness|               1.0|
+-------+--------------------+-------------+------------------+



## Profilers

In [32]:
profile = ColumnProfilerRunner(spark) \
                        .onData(df.select('TITULO_ORIGINAL', 'PUBLICO')) \
                        .run()

In [36]:
for key, value in profile.profiles.items():
    print(key)
    print(value)

TITULO_ORIGINAL
StandardProfiles for column: TITULO_ORIGINAL: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 150,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 0,
        "Unknown": 0,
        "String": 141350
    },
    "histogram": null
}
PUBLICO
NumericProfiles for column: PUBLICO: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 582,
    "dataType": "Integral",
    "isDataTypeInferred": false,
    "typeCounts": {},
    "histogram": null,
    "kll": "None",
    "mean": 44.088970640254686,
    "maximum": 906.0,
    "minimum": 1.0,
    "sum": 6231976.0,
    "stdDev": 61.57025735506737,
    "approxPercentiles": []
}


## Checks

In [39]:
test = Check(spark, CheckLevel.Warning, 'First check')

In [41]:
test1 = (
    VerificationSuite(spark)
    .onData(df)
    .addCheck(
        test.isUnique('PUBLICO', hint='Falha quando há valores iguais')
        .isNonNegative('PUBLICO', hint='Falha quando há valores negativos')
    ).run()
)

In [42]:
resultado_test1 = VerificationResult.checkResultsAsDataFrame(spark, test1)
resultado_test1.show()

+-----------+-----------+------------+--------------------+-----------------+--------------------+
|      check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+-----------+-----------+------------+--------------------+-----------------+--------------------+
|First check|    Warning|     Warning|UniquenessConstra...|          Failure|Value: 6.29642730...|
|First check|    Warning|     Warning|UniquenessConstra...|          Failure|Value: 6.29642730...|
|First check|    Warning|     Warning|ComplianceConstra...|          Success|                    |
+-----------+-----------+------------+--------------------+-----------------+--------------------+



/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [43]:
def verifica_acuracia(spark, df):
    tamanho = 150000
    teste_generico = Check(spark, CheckLevel.Error, 'Acuracia')
    teste = (VerificationSuite(spark)
        .onData(df)
        .addCheck(
            teste_generico.hasSize(lambda size: size >= 0.5 * tamanho and size <= 2.5 * tamanho)
        )
        .run()
    )
    return teste.checkResults[0]

In [44]:
resultado_acuracia = verifica_acuracia(spark, df)

Python Callback server started!


In [45]:
resultado_acuracia

{'check_status': 'Success',
 'check_level': 'Error',
 'constraint_status': 'Success',
 'check': 'Acuracia',
 'constraint_message': '',
 'constraint': 'SizeConstraint(Size(None))'}

In [46]:
siglas_estados = [
    "AC",  # Acre
    "AL",  # Alagoas
    "AP",  # Amapá
    "AM",  # Amazonas
    "BA",  # Bahia
    "CE",  # Ceará
    "DF",  # Distrito Federal
    "ES",  # Espírito Santo
    "GO",  # Goiás
    "MA",  # Maranhão
    "MT",  # Mato Grosso
    "MS",  # Mato Grosso do Sul
    "MG",  # Minas Gerais
    "PA",  # Pará
    "PB",  # Paraíba
    "PR",  # Paraná
    "PE",  # Pernambuco
    "PI",  # Piauí
    "RJ",  # Rio de Janeiro
    "RN",  # Rio Grande do Norte
    "RS",  # Rio Grande do Sul
    "RO",  # Rondônia
    "RR",  # Roraima
    "SC",  # Santa Catarina
    "SP",  # São Paulo
    "SE",  # Sergipe
    "TO"   # Tocantins
]

In [47]:
def cria_check(spark):
    return Check(spark, CheckLevel.Error, 'Teste Generico')

In [48]:
def verifica_validade(spark, df, teste_generico):
    verificacao = (
        VerificationSuite(spark)
        .onData(df)
        .addCheck(
            teste_generico.isContainedIn('UF_SALA_COMPLEXO', siglas_estados)
        ).run()
    )

    return verificacao.checkResults[0]

In [49]:
def verifica_validade(spark, df, teste_generico):
    siglas_estados = [
    "AC",  # Acre
    "AL",  # Alagoas
    "AP",  # Amapá
    "AM",  # Amazonas
    "BA",  # Bahia
    "CE",  # Ceará
    "DF",  # Distrito Federal
    "ES",  # Espírito Santo
    "GO",  # Goiás
    "MA",  # Maranhão
    "MT",  # Mato Grosso
    "MS",  # Mato Grosso do Sul
    "MG",  # Minas Gerais
    "PA",  # Pará
    "PB",  # Paraíba
    "PR",  # Paraná
    "PE",  # Pernambuco
    "PI",  # Piauí
    "RJ",  # Rio de Janeiro
    "RN",  # Rio Grande do Norte
    "RS",  # Rio Grande do Sul
    "RO",  # Rondônia
    "RR",  # Roraima
    "SC",  # Santa Catarina
    "SP",  # São Paulo
    "SE",  # Sergipe
    "TO"   # Tocantins
    ]
    verificacao = (
        VerificationSuite(spark)
        .onData(df)
        .addCheck(
            teste_generico.isContainedIn('UF_SALA_COMPLEXO', siglas_estados)
        ).run()
    )

    return verificacao.checkResults[0]

In [50]:
resultado_validade = verifica_validade(spark, df, cria_check(spark))

In [51]:
resultado_validade

{'check_status': 'Success',
 'check_level': 'Error',
 'constraint_status': 'Success',
 'check': 'Teste Generico',
 'constraint_message': '',
 'constraint': "ComplianceConstraint(Compliance(UF_SALA_COMPLEXO contained in AC,AL,AP,AM,BA,CE,DF,ES,GO,MA,MT,MS,MG,PA,PB,PR,PE,PI,RJ,RN,RS,RO,RR,SC,SP,SE,TO,`UF_SALA_COMPLEXO` IS NULL OR `UF_SALA_COMPLEXO` IN ('AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS','MG','PA','PB','PR','PE','PI','RJ','RN','RS','RO','RR','SC','SP','SE','TO'),None))"}

In [52]:
def verifica_completude(spark, df, teste_generico):
    verificacao = (
        VerificationSuite(spark)
        .onData(df)
        .addCheck(
            teste_generico.haveCompleteness(df.columns, lambda comp : comp >= 0.9)
        ).run()
    )
    return verificacao.checkResults[0]

In [53]:
resultado_completude = verifica_completude(spark, df, cria_check(spark))

In [54]:
resultado_completude

{'check_status': 'Error',
 'check_level': 'Error',
 'constraint_status': 'Failure',
 'check': 'Teste Generico',
 'constraint_message': 'Value: 0.6618747789175805 does not meet the constraint requirement!',
 'constraint': 'ComplianceConstraint(Compliance(Combined Completeness,(((((((((((((((DATA_EXIBICAO IS NOT NULL) AND (TITULO_ORIGINAL IS NOT NULL)) AND (TITULO_BRASIL IS NOT NULL)) AND (CPB_ROE IS NOT NULL)) AND (PAIS_OBRA IS NOT NULL)) AND (REGISTRO_SALA IS NOT NULL)) AND (NOME_SALA IS NOT NULL)) AND (PUBLICO IS NOT NULL)) AND (REGISTRO_GRUPO_EXIBIDOR IS NOT NULL)) AND (REGISTRO_EXIBIDOR IS NOT NULL)) AND (REGISTRO_COMPLEXO IS NOT NULL)) AND (MUNICIPIO_SALA_COMPLEXO IS NOT NULL)) AND (UF_SALA_COMPLEXO IS NOT NULL)) AND (RAZAO_SOCIAL_DISTRIBUIDORA IS NOT NULL)) AND (CNPJ_DISTRIBUIDORA IS NOT NULL)),None))'}

In [55]:
def  verifica_unicidade(spark, df, teste_generico, coluna):
    verificacao = (
        VerificationSuite(spark)
        .onData(df)
        .addCheck(
            teste_generico.isUnique(coluna)
        ).run()
    )
    return verificacao.checkResults[0]

In [56]:
verifica_unicidade(spark, df, cria_check(spark), 'PUBLICO')

{'check_status': 'Error',
 'check_level': 'Error',
 'constraint_status': 'Failure',
 'check': 'Teste Generico',
 'constraint_message': 'Value: 6.29642730810046E-4 does not meet the constraint requirement!',
 'constraint': 'UniquenessConstraint(Uniqueness(List(PUBLICO),None))'}

In [57]:
from pydeequ.suggestions import *

In [58]:
sugestoes = (
    ConstraintSuggestionRunner(spark)
    .onData(df)
    .addConstraintRule(DEFAULT())
    .run()
)

In [59]:
for sugestao in sugestoes['constraint_suggestions']:
    print(f"Sugestão : {sugestao['column_name']} : {sugestao['description']}")

Sugestão : REGISTRO_COMPLEXO : 'REGISTRO_COMPLEXO' has no negative values
Sugestão : REGISTRO_COMPLEXO : 'REGISTRO_COMPLEXO' has less than 1% missing values
Sugestão : REGISTRO_COMPLEXO : 'REGISTRO_COMPLEXO' has type Integral
Sugestão : UF_SALA_COMPLEXO : 'UF_SALA_COMPLEXO' has value range 'SP', 'RJ', 'MG', 'PR', 'RS', 'SC', 'BA', 'PE', 'GO', 'CE', 'DF', 'ES', 'PA', 'MT', 'AM', 'MA', 'PB', 'AL', 'RN', 'SE', 'MS', 'RO', 'PI', 'TO', 'AP', 'RR', 'AC'
Sugestão : UF_SALA_COMPLEXO : 'UF_SALA_COMPLEXO' is not null
Sugestão : UF_SALA_COMPLEXO : 'UF_SALA_COMPLEXO' has value range 'SP', 'RJ', 'MG', 'PR', 'RS', 'SC', 'BA', 'PE', 'GO', 'CE', 'DF', 'ES', 'PA', 'MT', 'AM' for at least 90.0% of values
Sugestão : REGISTRO_SALA : 'REGISTRO_SALA' has no negative values
Sugestão : REGISTRO_SALA : 'REGISTRO_SALA' has less than 1% missing values
Sugestão : REGISTRO_SALA : 'REGISTRO_SALA' has type Integral
Sugestão : PAIS_OBRA : 'PAIS_OBRA' is not null
Sugestão : PAIS_OBRA : 'PAIS_OBRA' has value range 'EST

In [60]:
for sugestao in sugestoes['constraint_suggestions']:
    print(f"Sugestão : {sugestao['column_name']} : {sugestao['description']}")
    print(f"teste sugerido: {sugestao['code_for_constraint']}")

Sugestão : REGISTRO_COMPLEXO : 'REGISTRO_COMPLEXO' has no negative values
teste sugerido: .isNonNegative("REGISTRO_COMPLEXO")
Sugestão : REGISTRO_COMPLEXO : 'REGISTRO_COMPLEXO' has less than 1% missing values
teste sugerido: .hasCompleteness("REGISTRO_COMPLEXO", lambda x: x >= 0.99, "It should be above 0.99!")
Sugestão : REGISTRO_COMPLEXO : 'REGISTRO_COMPLEXO' has type Integral
teste sugerido: .hasDataType("REGISTRO_COMPLEXO", ConstrainableDataTypes.Integral)
Sugestão : UF_SALA_COMPLEXO : 'UF_SALA_COMPLEXO' has value range 'SP', 'RJ', 'MG', 'PR', 'RS', 'SC', 'BA', 'PE', 'GO', 'CE', 'DF', 'ES', 'PA', 'MT', 'AM', 'MA', 'PB', 'AL', 'RN', 'SE', 'MS', 'RO', 'PI', 'TO', 'AP', 'RR', 'AC'
teste sugerido: .isContainedIn("UF_SALA_COMPLEXO", ["SP", "RJ", "MG", "PR", "RS", "SC", "BA", "PE", "GO", "CE", "DF", "ES", "PA", "MT", "AM", "MA", "PB", "AL", "RN", "SE", "MS", "RO", "PI", "TO", "AP", "RR", "AC"])
Sugestão : UF_SALA_COMPLEXO : 'UF_SALA_COMPLEXO' is not null
teste sugerido: .isComplete("UF_SA